In [1]:
import pandas as pd
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go

In [2]:
def fetch_data():
    file_path = "companies_esg_scores.xlsx"
    df = pd.read_excel(file_path, header = 0)
    return df

In [3]:
e_metrics = ["Emission_intensities", "Energy_consumption_intensity", "Waste_generated", "Water_intensity"]
s_metrics = ["Average_training_hours_per_employee", "Current_employees_by_age_groups", "Current_employees_by_gender",
             "Employee_satisfaction_rate", "New_hires_by_gender", "New_hires_by_age",
             "Total_turnover", "Turnover_by_gender", "Turnover_by_age", "Fatalities",
             "Percentage_of_employees_covered_by_health_insurance", "High_consequence_injuries",
             "Work_related_injuries", "Company_donated"]
g_metrics = ["Board_independence", "Women_in_the_management_team", "Women_on_the_board"]

In [4]:
# Create a Dash app
app = Dash(__name__)

In [5]:
# Initialize the layout
app.layout = html.Div([
    # E, S, G dropdowns
    # E dropdown
    html.Div([
        html.Label("Select E Indicators"),
        dcc.Dropdown(
            id = "e-dropdown",
            options = [{"label": metric, "value": metric} for metric in e_metrics],
            multi = True,
            placeholder = "Select E indicators"
        ),
    ], style = {"width": "30%", "display": "inline-block", "padding": "10px"}),

    # S dropdown
    html.Div([
        html.Label("Select S Indicators"),
        dcc.Dropdown(
            id = "s-dropdown",
            options = [{"label": metric, "value": metric} for metric in s_metrics],
            multi = True,
            placeholder = "Select S indicators"
        ),
    ], style = {"width": "30%", "display": "inline-block", "padding": "10px"}),
    
    # G dropdown
    html.Div([
        html.Label("Select G Indicators"),
        dcc.Dropdown(
            id = "g-dropdown",
            options = [{"label": metric, "value": metric} for metric in g_metrics],
            multi = True,
            placeholder = "Select G indicators"
        ),
    ], style = {"width": "30%", "display": "inline-block", "padding": "10px"}),

    # Output table and chart
    html.Div(id = "output-table", style = {"marginTop": "20px"}),
    dcc.Graph(id = "output-graph", style = {"height": "400px", "marginTop": "20px"}),
    dcc.Input(id = "dummy-input", type = "hidden", value = "dummy") 
])

# callback function
@app.callback(
    [Output("output-table", "children"), Output("output-graph", "figure")],
    [Input("e-dropdown", "value"), Input("s-dropdown", "value"), Input("g-dropdown", "value")]
)
def update_output(e_selection, s_selection, g_selection):
    df = fetch_data()
    selected_data = {}

    if e_selection:
        for indicator in e_selection:
            selected_data[indicator] = df[indicator].iloc[-1]
    if s_selection:
        for indicator in s_selection:
            selected_data[indicator] = df[indicator].iloc[-1]
    if g_selection:
        for indicator in g_selection:
            selected_data[indicator] = df[indicator].iloc[-1]
    
    if not selected_data:
        return [], {}

    # Create the table
    table_data = pd.DataFrame(list(selected_data.items()), columns = ["Indicator", "Score"])

    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in table_data.columns])),
        html.Tbody([
            html.Tr([html.Td(row[col]) for col in table_data.columns]) for _, row in table_data.iterrows()
        ])
    ])

    # Create the barplot
    fig = go.Figure()
    for indicator, score in selected_data.items():
        if indicator in e_metrics:
            color = "lightgreen"  
        elif indicator in s_metrics:
            color = "lightblue"   
        elif indicator in g_metrics:
            color = "green"       
        fig.add_trace(go.Bar(
            x = [indicator],
            y = [score],
            name = indicator, 
            marker = dict(color = color)
        ))
        
    fig.update_layout(
        title = "ESG Indicator Scores",
        yaxis_title = "Score",
        xaxis = dict(tickangle = 45),
        barmode = "group",
        font = dict(family = "Times New Roman", size = 16),
        plot_bgcolor = "white",  
        paper_bgcolor = "white" 
    )

    return table, fig

In [6]:
if __name__ == "__main__":
    app.run_server(debug = True, port = 8070)